In [ ]:
# Join Expressions
# A join brings together two sets of data, the left and the right, by comparing the value of one or
# more keys of the left and right and evaluating the result of a join expression that determines
# whether Spark should bring together the left set of data with the right set of data. The most
# common join expression, an equi-join, compares whether the specified keys in your left and
# right datasets are equal. If they are equal, Spark will combine the left and right datasets. The
# opposite is true for keys that do not match; Spark discards the rows that do not have matching
# keys. Spark also allows for much more sophsticated join policies in addition to equi-joins. We
# can even use complex types and perform something like checking whether a key exists within an
# array when you perform a join.

In [ ]:
# Join Types
# Whereas the join expression determines whether two rows should join, the join type determines
# what should be in the result set. There are a variety of different join types available in Spark for you to use:
#     Inner joins (keep rows with keys that exist in the left and right datasets)
#     Outer joins (keep rows with keys in either the left or right datasets)
#     Left outer joins (keep rows with keys in the left dataset)
#     Right outer joins (keep rows with keys in the right dataset)
#     Left semi joins (keep the rows in the left, and only the left, dataset where the key appears in the right dataset)
#     Left anti joins (keep the rows in the left, and only the left, dataset where they do not appear in the right dataset) 
#     Natural joins (perform a join by implicitly matching the columns between the two datasets with the same names)
#     Cross (or Cartesian) joins (match every row in the left dataset with every row in the right dataset)

In [4]:
# importing session from sql from pyspark to start the sessio
from pyspark.sql import SparkSession

In [5]:
# creating the seasion
spark = SparkSession.builder.appName("Joins").getOrCreate()

22/10/18 16:41:11 WARN Utils: Your hostname, HP-G62 resolves to a loopback address: 127.0.1.1; using 192.168.18.113 instead (on interface enp3s0)
22/10/18 16:41:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/18 16:41:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/18 16:41:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/18 16:41:14 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/10/18 16:41:14 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/10/18 16:41:14 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [8]:
person = spark.createDataFrame([
    (0, "Bill Chambers", 0, [100]),
    (1, "Matei Zaharia", 1, [500, 250, 100]),
    (2, "Michael Armbrust", 1, [250, 100])])\
    .toDF("id", "name", "graduate_program", "spark_status")

graduateProgram = spark.createDataFrame([
    (0, "Masters", "School of Information", "UC Berkeley"),
    (2, "Masters", "EECS", "UC Berkeley"),
    (1, "Ph.D.", "EECS", "UC Berkeley")])\
    .toDF("id", "degree", "department", "school")

sparkStatus = spark.createDataFrame([
    (500, "Vice President"),
    (250, "PMC Member"),
    (100, "Contributor")])\
    .toDF("id", "status")

In [10]:
person.show()

+---+----------------+----------------+---------------+
| id|            name|graduate_program|   spark_status|
+---+----------------+----------------+---------------+
|  0|   Bill Chambers|               0|          [100]|
|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  2|Michael Armbrust|               1|     [250, 100]|
+---+----------------+----------------+---------------+



In [11]:
graduateProgram.show()

+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  2|Masters|                EECS|UC Berkeley|
|  1|  Ph.D.|                EECS|UC Berkeley|
+---+-------+--------------------+-----------+



In [12]:
sparkStatus.show()

+---+--------------+
| id|        status|
+---+--------------+
|500|Vice President|
|250|    PMC Member|
|100|   Contributor|
+---+--------------+



In [13]:
# let’s register these as tables so that we use them throughout the chapter:
person.createOrReplaceTempView("person")
graduateProgram.createOrReplaceTempView("graduateProgram")
sparkStatus.createOrReplaceTempView("sparkStatus")

In [ ]:
# Inner Joins
# in SQL
# SELECT * FROM person JOIN graduateProgram
# ON person.graduate_program = graduateProgram.id

# Inner joins evaluate the keys in both of the DataFrames or tables and include (and join together)
# only the rows that evaluate to true. In the following example, we join the graduateProgram
#     person.join(graduateProgram, joinExpression, joinType).show()
    
# DataFrame with the person DataFrame to create a new DataFrame:

In [15]:
# Keys that do not exist in both DataFrames will not show in the resulting DataFrame. For
# example, the following expression would result in zero values in the resulting DataFrame:

wrongJoinExpression = person["name"] == graduateProgram["school"]

In [16]:
wrongJoinExpression.show()

TypeError: 'Column' object is not callable

In [17]:
# Inner joins are the default join, so we just need to specify our left DataFrame and join the right in the JOIN expression:
person.join(graduateProgram, person["graduate_program"] == graduateProgram['id'],"inner").show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+



In [18]:
# Inner joins are the default join, so we just need to specify our left DataFrame and join the right in the JOIN expression:
joinExpression = person["graduate_program"] == graduateProgram['id']
joinType="inner"
person.join(graduateProgram, joinExpression,joinType).show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+



In [19]:
# Outer Joins
# in SQL
# SELECT * FROM person FULL OUTER JOIN graduateProgram
# ON graduate_program = graduateProgram.id

# Outer joins evaluate the keys in both of the DataFrames or tables and includes (and joins together) the rows that
# evaluate to true or false. If there is no equivalent row in either the left or right DataFrame, Spark will insert null:

#     person.join(graduateProgram, joinExpression, joinType).show()
joinExpression = person["graduate_program"] == graduateProgram['id']
joinType="outer"
person.join(graduateProgram, joinExpression, joinType).show()

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|null|            null|            null|           null|  2|Masters|                EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+



In [20]:
# Left Outer Joins
# in SQL
# SELECT * FROM graduateProgram LEFT OUTER JOIN person
# ON person.graduate_program = graduateProgram.id

# Left outer joins evaluate the keys in both of the DataFrames or tables and includes all rows from
# the left DataFrame as well as any rows in the right DataFrame that have a match in the left
# DataFrame. If there is no equivalent row in the right DataFrame, Spark will insert null:
    
joinExpression = person["graduate_program"] == graduateProgram['id']
joinType = "left_outer"
graduateProgram.join(person, joinExpression, joinType).show()


+---+-------+--------------------+-----------+----+----------------+----------------+---------------+
| id| degree|          department|     school|  id|            name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+----+----------------+----------------+---------------+
|  0|Masters|School of Informa...|UC Berkeley|   0|   Bill Chambers|               0|          [100]|
|  2|Masters|                EECS|UC Berkeley|null|            null|            null|           null|
|  1|  Ph.D.|                EECS|UC Berkeley|   2|Michael Armbrust|               1|     [250, 100]|
|  1|  Ph.D.|                EECS|UC Berkeley|   1|   Matei Zaharia|               1|[500, 250, 100]|
+---+-------+--------------------+-----------+----+----------------+----------------+---------------+



In [21]:
# Right Outer Joins
# in SQL
# SELECT * FROM person RIGHT OUTER JOIN graduateProgram
# ON person.graduate_program = graduateProgram.id

# Right outer joins evaluate the keys in both of the DataFrames or tables and includes all rows
# from the right DataFrame as well as any rows in the left DataFrame that have a match in the right
# DataFrame. If there is no equivalent row in the left DataFrame, Spark will insert null:

joinExpression = person["graduate_program"] == graduateProgram['id']
joinType = "right_outer"
person.join(graduateProgram, joinExpression, joinType).show()


+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|null|            null|            null|           null|  2|Masters|                EECS|UC Berkeley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+



In [22]:
# Left Semi Joins
# in SQL
# SELECT * FROM gradProgram2 LEFT SEMI JOIN person
# ON gradProgram2.id = person.graduate_program

# Semi joins are a bit of a departure from the other joins. They do not actually include any values
# from the right DataFrame. They only compare values to see if the value exists in the second
# DataFrame. If the value does exist, those rows will be kept in the result, even if there are
# duplicate keys in the left DataFrame. Think of left semi joins as filters on a DataFrame, as
# opposed to the function of a conventional join:

joinExpression = person["graduate_program"] == graduateProgram['id']
joinType = "left_semi"
graduateProgram.join(person, joinExpression, joinType).show()

+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  1|  Ph.D.|                EECS|UC Berkeley|
+---+-------+--------------------+-----------+



In [23]:
gradProgram2 = graduateProgram.union(spark.createDataFrame([
(0, "Masters", "Duplicated Row", "Duplicated School")]))
gradProgram2.createOrReplaceTempView("gradProgram2")
gradProgram2.join(person, joinExpression, joinType).show()

+---+-------+--------------------+-----------------+
| id| degree|          department|           school|
+---+-------+--------------------+-----------------+
|  0|Masters|School of Informa...|      UC Berkeley|
|  1|  Ph.D.|                EECS|      UC Berkeley|
|  0|Masters|      Duplicated Row|Duplicated School|
+---+-------+--------------------+-----------------+



In [24]:
# Left Anti Joins
# in SQL
# SELECT * FROM graduateProgram LEFT ANTI JOIN person
# ON graduateProgram.id = person.graduate_program

# Left anti joins are the opposite of left semi joins. Like left semi joins, they do not actually
# include any values from the right DataFrame. They only compare values to see if the value exists
# in the second DataFrame. However, rather than keeping the values that exist in the second
# DataFrame, they keep only the values that do not have a corresponding key in the second
# DataFrame. Think of anti joins as a NOT IN SQL-style filter:

joinExpression = person["graduate_program"] == graduateProgram['id']
joinType = "left_anti"
graduateProgram.join(person, joinExpression, joinType).show()

+---+-------+----------+-----------+
| id| degree|department|     school|
+---+-------+----------+-----------+
|  2|Masters|      EECS|UC Berkeley|
+---+-------+----------+-----------+



In [ ]:
# Natural Joins
# in SQL
# SELECT * FROM graduateProgram NATURAL JOIN person

# Natural joins make implicit guesses at the columns on which you would like to join. It finds
# matching columns and returns the results. Left, right, and outer natural joins are all supported.

# WARNING
# Implicit is always dangerous! The following query will give us incorrect results because the two
# DataFrames/tables share a column name (id), but it means different things in the datasets. You should
# always use this join with caution.

In [25]:
# Cross (Cartesian) Joins
# in SQL
# SELECT * FROM graduateProgram CROSS JOIN person
# ON graduateProgram.id = person.graduate_program

# The last of our joins are cross-joins or cartesian products. Cross-joins in simplest terms are inner
# joins that do not specify a predicate. Cross joins will join every single row in the left DataFrame
# to ever single row in the right DataFrame. This will cause an absolute explosion in the number of
# rows contained in the resulting DataFrame. If you have 1,000 rows in each DataFrame, the crossjoin of these will result in 1,000,000 (1,000 x 1,000) rows. For this reason, you must very
# explicitly state that you want a cross-join by using the cross join keyword:
    
joinExpression = person["graduate_program"] == graduateProgram['id']
joinType = "cross"
graduateProgram.join(person, joinExpression, joinType).show()

+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
| id| degree|          department|     school| id|            name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
|  0|Masters|School of Informa...|UC Berkeley|  0|   Bill Chambers|               0|          [100]|
|  1|  Ph.D.|                EECS|UC Berkeley|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  1|  Ph.D.|                EECS|UC Berkeley|  2|Michael Armbrust|               1|     [250, 100]|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+



In [27]:
# If you truly intend to have a cross-join, you can call that out explicitly:

# in SQL
# SELECT * FROM graduateProgram CROSS JOIN person

person.crossJoin(graduateProgram).show()

# WARNING
# You should use cross-joins only if you are absolutely, 100 percent sure that this is the join you need.
# There is a reason why you need to be explicit when defining a cross-join in Spark. They’re dangerous!
# Advanced users can set the session-level configuration spark.sql.crossJoin.enable to true in
# order to allow cross-joins without warnings or without Spark trying to perform another join for you.

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  0|   Bill Chambers|               0|          [100]|  2|Masters|                EECS|UC Berkeley|
|  0|   Bill Chambers|               0|          [100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  2|Masters|                EECS|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  0|Masters|School of Informa...|UC 